<a href="https://colab.research.google.com/github/iamRahulB/Pytorch-practice/blob/main/RNN/LSTM_on_text_with_pad_sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch==2.2.0 torchvision torchaudio torchtext==0.17.0 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.7/811.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U portalocker==2.8.2

In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [4]:
df=pd.read_csv('https://github.com/LawrenceDuan/IMDb-Review-Analysis/raw/refs/heads/master/IMDb_Reviews.csv')

In [5]:


# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['cleaned_review'] = df['review'].apply(preprocess_text)

df.head(5)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,review,sentiment,cleaned_review
0,My family and I normally do not watch local mo...,1,family normally watch local movie simple reaso...
1,"Believe it or not, this was at one time the wo...",0,believe one time worst movie ever seen since t...
2,"After some internet surfing, I found the ""Home...",0,internet surfing found homefront series dvd io...
3,One of the most unheralded great works of anim...,1,one unheralded great work animation though mak...
4,"It was the Sixties, and anyone with long hair ...",0,sixty anyone long hair hip distant attitude co...


In [6]:
new_df=df.sample(2000)

In [7]:
len(new_df)

2000

In [8]:
new_df.drop('review',inplace=True,axis=1)

In [9]:
new_df.head(5)

,sentiment,cleaned_review
19880,1,became fan tv series homicide life street late...
43781,1,many kind reunion show one kind old actor take...
24844,0,horribly dated must insistplease remake frankl...
42114,0,scenery pretty dog cute film absolutely nothin...
28410,0,harmony korine im sure trying film turn good d...


In [10]:
import torch
from torch import nn
from torchtext import datasets
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.model_selection import train_test_split

In [11]:
train_data,test_data=train_test_split(new_df, test_size=0.2,random_state=42)

In [12]:
print(train_data.shape, test_data.shape)

train_data.head(3)

(1600, 2) (400, 2)


,sentiment,cleaned_review
49316,0,three minute thing started fastforwarding paus...
44665,1,really enjoyed thriller wanted see soon cast e...
31712,1,incredible little english film many reason fir...


In [13]:
from torch.utils.data import Dataset,DataLoader

In [14]:
class CustomDataset(Dataset):
    def __init__(self,texts,targets):
        super().__init__()
        self.texts=texts
        self.targets=targets


    def __len__(self):
        return len(self.texts)   # or can be len(self.targets) as its a list

    def __getitem__(self,idx):
        item_text=self.texts[idx]
        item_target=self.targets[idx]
        return item_target,item_text



In [15]:
train_iter=CustomDataset(train_data['cleaned_review'].tolist(),train_data['sentiment'].tolist())
test_iter=CustomDataset(test_data['cleaned_review'].tolist(),test_data['sentiment'].tolist())

In [16]:
next(iter(train_iter)),next(iter(test_iter)), len(train_iter),len(test_iter),

((0,
  'three minute thing started fastforwarding pausing nudity bad movie always include good looking woman ten minute done wishing could get money back rental store people write movie sanctioned mpaa come writer bad guy always get car bomb activated good guy remote control thats way done since day ottoman empire also add insult injury twist end formulaic could come action movie written past year burt reynolds fine concentrate real moviesbr br movie waste time run away run away'),
 (1,
  'first saw breaking glass released england loved caught august canadian station still great thing regret cant find sound track dvd storesanyone care shed light must order priced internet company getting back film music stand test time hazelkate something say britainactually decade moved canada reason one thatcher country time please get chance watch movie wont sorry'),
 1600,
 400)

In [17]:
tokenizer=get_tokenizer('basic_english')

In [18]:
tokenizer("hel hel")

['hel', 'hel']

In [19]:
def build_vocab(data):

    for label, text in data:
        yield tokenizer(text)



vocab=build_vocab_from_iterator(build_vocab(train_iter),specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])


In [20]:
vocab['god'],len(vocab)

(373, 24500)

In [21]:
text_pipe=lambda x: vocab(tokenizer(x))
label_pipe=lambda x : int(x)

In [22]:
text_pipe("he might be  thinking i am good"), label_pipe(8)

([131, 169, 8500, 397, 0, 0, 6], 8)

In [23]:
from torch.nn.utils.rnn import pad_sequence

In [24]:
def collate_batch(batch):
    all_texts=[]
    all_labels=[]
    for label, text in batch:
        preprocessed_text=torch.tensor(text_pipe(text),dtype=torch.int64)
        all_texts.append(preprocessed_text)
        all_labels.append(label_pipe(label))

    padded_texts = pad_sequence(all_texts, batch_first=True, padding_value=0)
    all_labels = torch.tensor(all_labels, dtype=torch.int64)

    return all_labels,padded_texts


### Concatenated Text Tensor
- All text sequences in a batch are concatenated into a single tensor using `torch.cat`.
- **Example**: If you have three sequences `[seq1, seq2, seq3]`, the concatenated tensor might look like `[seq1_tokens, seq2_tokens, seq3_tokens]`.

### Offsets Tensor
- The `offsets` tensor indicates the starting index of each sequence within the concatenated tensor.
- **Example**: If `seq1` has 5 tokens, `seq2` has 3 tokens, and `seq3` has 4 tokens, the `offsets` tensor would be `[0, 5, 8]`.

### EmbeddingBag Layer
- The `nn.EmbeddingBag` layer in PyTorch is designed to handle such concatenated sequences efficiently.
- It uses the `offsets` tensor to know where each sequence starts and computes embeddings for each sequence accordingly.

### Model Forward Pass
- During the forward pass, the model processes the concatenated text tensor and uses the `offsets` tensor to correctly segment the sequences.
- The `nn.EmbeddingBag` layer aggregates the embeddings for each sequence based on the offsets.

### Output Layer
- The model’s output layer (e.g., a fully connected layer) then takes these aggregated embeddings and produces a prediction for each sequence.
- The predictions correspond to the labels of the individual text sequences.


In [25]:
label_list,text_list=collate_batch([
    (0, "This is the first text."),
    (1, "Here is another text."),
    (0, "And a third one.")
])

In [26]:
label_list, text_list

(tensor([0, 1, 0]),
 tensor([[   0,    0,    0,   23, 4510,    0],
         [1856,    0,   82, 4510,    0,    0],
         [   0,    0,  558,    4,    0,    0]]))

In [27]:
train_loader=DataLoader(train_iter,batch_size=32,collate_fn=collate_batch, shuffle=True,drop_last=True)
test_loader=DataLoader(test_iter,batch_size=32,collate_fn=collate_batch, shuffle=False,drop_last=True)

In [28]:
next(iter(train_loader))

(tensor([1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         1, 0, 0, 1, 1, 0, 1, 1]),
 tensor([[  267, 13444,   974,  ...,     0,     0,     0],
         [  129,  8093, 17589,  ...,     0,     0,     0],
         [ 3359,    59,     2,  ...,     0,     0,     0],
         ...,
         [    4,   207,  4549,  ...,     0,     0,     0],
         [ 2245,   357,  1535,  ...,     0,     0,     0],
         [  382,    11,  2680,  ...,     0,     0,     0]]))

In [29]:
from torch import nn

In [62]:

class EnhancedModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, dropout=0.5):
        super(EnhancedModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, 1)  # *2 for bidirectional

    def forward(self, text):
        # Get the embeddings for the input text
        embedded = self.embedding(text)

        # Pass the embeddings through the LSTM
        lstm_out, (hn, cn) = self.lstm(embedded)

        # Use the hidden state of the last LSTM layer as the sequence representation
        # hn has shape (num_layers * num_directions, batch_size, hidden_dim)
        # We take the last layer's hidden state from both directions: hn[-2] and hn[-1]
        final_hidden_state = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim=1)

        # Apply dropout
        final_hidden_state = self.dropout(final_hidden_state)

        # Pass the final hidden state through the fully connected layer
        output = self.fc(final_hidden_state)

        return output
# Example usage
embed_dim = 300
hidden_dim = 128
num_layers = 2
model = EnhancedModel(len(vocab), embed_dim, hidden_dim, num_layers)



In [51]:
from sklearn.metrics import accuracy_score


In [58]:
# for idx, (label, text, offsets) in enumerate(train_loader):
#     print(idx,label,text,offsets)
#     print(model(text,offsets))
#     break

In [63]:
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)

# Set the random seed before the loop
torch.manual_seed(42)

# Assuming loss_fn is nn.BCEWithLogitsLoss for binary classification

# Move the model to the GPU
model = model.to(device)

# Training loop
epochs = 30

for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_acc = 0
    model.train()

    for batch, (y_train, X_train) in enumerate(train_loader):
        # Move data to the GPU
        X_train = X_train.to(device)
        y_train = y_train.to(device)

        # Forward pass
        y_logits = model(X_train)

        # Calculate loss
        y_train = y_train.float().unsqueeze(1)
        train_batch_loss = loss_fn(y_logits, y_train)

        # Apply sigmoid and threshold to get binary predictions
        train_probs = torch.sigmoid(y_logits)
        train_batch_preds = (train_probs > 0.5).float()

        # Calculate accuracy using sklearn
        train_batch_acc = accuracy_score(y_train.cpu().numpy(), train_batch_preds.cpu().numpy())

        # Backpropagation and optimization
        optimizer.zero_grad()
        train_batch_loss.backward()
        optimizer.step()

        train_loss += train_batch_loss.item()
        train_acc += train_batch_acc

    model.eval()
    train_acc /= len(train_loader)
    train_loss /= len(train_loader)
    test_loss = 0
    test_acc = 0

    with torch.inference_mode():
        for y_test,X_test in test_loader:
            # Move data to the GPU
            X_test = X_test.to(device)
            y_test = y_test.to(device)

            # Forward pass
            y_test_logits = model(X_test)

            # Calculate loss
            y_test = y_test.float().unsqueeze(1)
            test_batch_loss = loss_fn(y_test_logits, y_test)

            # Apply sigmoid and threshold to get binary predictions
            test_probs = torch.sigmoid(y_test_logits)
            test_batch_preds = (test_probs > 0.5).float()

            # Calculate accuracy using sklearn
            test_batch_acc = accuracy_score(y_test.cpu().numpy(), test_batch_preds.cpu().numpy())

            test_loss += test_batch_loss.item()
            test_acc += test_batch_acc

    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Train ACC: {train_acc*100:.2f} | Test Loss: {test_loss:.4f} | Test ACC: {test_acc*100:.2f}")

print("Training complete.")





  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.6934 | Train ACC: 51.25 | Test Loss: 0.6991 | Test ACC: 47.92
Epoch: 2 | Train Loss: 0.5941 | Train ACC: 68.62 | Test Loss: 0.6320 | Test ACC: 65.89
Epoch: 3 | Train Loss: 0.3630 | Train ACC: 86.00 | Test Loss: 0.7755 | Test ACC: 65.62
Epoch: 4 | Train Loss: 0.1348 | Train ACC: 94.94 | Test Loss: 0.9493 | Test ACC: 64.32
Epoch: 5 | Train Loss: 0.0461 | Train ACC: 98.94 | Test Loss: 1.2131 | Test ACC: 67.97
Epoch: 6 | Train Loss: 0.0096 | Train ACC: 99.94 | Test Loss: 1.4376 | Test ACC: 67.97
Epoch: 7 | Train Loss: 0.0018 | Train ACC: 100.00 | Test Loss: 1.5699 | Test ACC: 69.01
Epoch: 8 | Train Loss: 0.0010 | Train ACC: 100.00 | Test Loss: 1.6592 | Test ACC: 70.05
Epoch: 9 | Train Loss: 0.0007 | Train ACC: 100.00 | Test Loss: 1.7305 | Test ACC: 70.31
Epoch: 10 | Train Loss: 0.0005 | Train ACC: 100.00 | Test Loss: 1.7911 | Test ACC: 70.31
Epoch: 11 | Train Loss: 0.0004 | Train ACC: 100.00 | Test Loss: 1.8328 | Test ACC: 70.31
Epoch: 12 | Train Loss: 0.0003 | Tra

In [292]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return (torch.sigmoid(output)>0.5).int()

test=list(test_iter)[7][1]
predict(test,text_pipe)

TypeError: Embedding.forward() takes 2 positional arguments but 3 were given

In [ ]:
list(test_iter)[7]


In [296]:
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, TensorDataset


In [310]:
texts = train_data['cleaned_review'].tolist()
labels = train_data['sentiment'].tolist()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=300)  # Limit to top 5000 features

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.int64)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.int64)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [311]:
class SimpleLstm(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100, hidden_dim=64):
        super(SimpleLstm, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.linear = nn.Linear(in_features=hidden_dim, out_features=1)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = self.linear(lstm_out[:, -1, :])
        return out


torch.manual_seed(42)
model = SimpleLstm(vocab_size=len(vocab))
model

SimpleLstm(
  (embedding): Embedding(54621, 100)
  (lstm): LSTM(100, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [312]:
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.01)

# Set the random seed before the loop
torch.manual_seed(42)

# Assuming loss_fn is nn.BCEWithLogitsLoss for binary classification

# Move the model to the GPU
model = model.to(device)

# Training loop
epochs = 3

for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_acc = 0
    model.train()

    for batch, (X_train, y_train) in enumerate(train_loader):
        # Move data to the GPU
        X_train = X_train.to(device)
        y_train = y_train.to(device)

        # Forward pass
        y_logits = model(X_train)

        # Calculate loss
        y_train = y_train.float().unsqueeze(1)
        train_batch_loss = loss_fn(y_logits, y_train)

        # Apply sigmoid and threshold to get binary predictions
        train_probs = torch.sigmoid(y_logits)
        train_batch_preds = (train_probs > 0.5).float()

        # Calculate accuracy using sklearn
        train_batch_acc = accuracy_score(y_train.cpu().numpy(), train_batch_preds.cpu().numpy())

        # Backpropagation and optimization
        optimizer.zero_grad()
        train_batch_loss.backward()
        optimizer.step()

        train_loss += train_batch_loss.item()
        train_acc += train_batch_acc

    model.eval()
    train_acc /= len(train_loader)
    train_loss /= len(train_loader)
    test_loss = 0
    test_acc = 0

    with torch.inference_mode():
        for X_test, y_test in test_loader:
            # Move data to the GPU
            X_test = X_test.to(device)
            y_test = y_test.to(device)

            # Forward pass
            y_test_logits = model(X_test)

            # Calculate loss
            y_test = y_test.float().unsqueeze(1)
            test_batch_loss = loss_fn(y_test_logits, y_test)

            # Apply sigmoid and threshold to get binary predictions
            test_probs = torch.sigmoid(y_test_logits)
            test_batch_preds = (test_probs > 0.5).float()

            # Calculate accuracy using sklearn
            test_batch_acc = accuracy_score(y_test.cpu().numpy(), test_batch_preds.cpu().numpy())

            test_loss += test_batch_loss.item()
            test_acc += test_batch_acc

    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Train ACC: {train_acc*100:.2f} | Test Loss: {test_loss:.4f} | Test ACC: {test_acc*100:.2f}")

print("Training complete.")





  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.6948 | Train ACC: 50.04 | Test Loss: 0.6935 | Test ACC: 50.09
Epoch: 2 | Train Loss: 0.6939 | Train ACC: 50.00 | Test Loss: 0.6938 | Test ACC: 49.91
Epoch: 3 | Train Loss: 0.6936 | Train ACC: 50.43 | Test Loss: 0.6933 | Test ACC: 50.09
Training complete.
